In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import cv2
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing
import numpy as np
import act
import sys
%matplotlib inline

In [3]:
def create_file(ds, row_wt, col_wt, remnant, clouds, precip, fog_flags, precip_flags, cloud_flags, clear_air_flag, cleaned_thresh, layers, overlap_function, max_height=5000):
    """
    Create a NetCDF file from the provided dataset and additional data.
    Parameters
    ----------
    ds : xarray.Dataset
        The dataset containing the main data.
    col_wt : xarray.DataArray
        Column weights to be added to the dataset.
    row_wt : xarray.DataArray
        Row weights to be added to the dataset.
    remnant : xarray.DataArray
        Remnant data to be added to the dataset.
    edges : xarray.DataArray
        Edges data to be added to the dataset.
    """


    clouds = xr.DataArray(
        clouds.astype(np.uint8),
        dims=('range', 'time'),
        coords={'range': ds['range'], 'time': ds['time']},
        attrs={
            'long_name': 'detected cloud regions',
            }
    )

    precip = xr.DataArray(
        precip.astype(np.uint8),
        dims=('range', 'time'),
        coords={'range': ds['range'], 'time': ds['time']},
        attrs={
            'long_name': 'detected precipitation regions',
            'comment': 'detected/not-detected (1/0)'
        }
    )
    

    fog_flags = xr.DataArray(
        fog_flags,
        dims=('time',),
        coords={'time': ds['time']},
        attrs={
            'long_name': 'detected fog times',
            'comment': 'detected/not-detected (1/0)'
        }
    )
    

    precip_flags = xr.DataArray(
        precip_flags,
        dims=('time',),
        coords={'time': ds['time']},
        attrs={
            'long_name': 'detected precipitation times',
            'comment': 'detected/not-detected (1/0)'
        }
    )
    
    cloud_flags = xr.DataArray(
        cloud_flags,
        dims=('time',),
        coords={'time': ds['time']},
        attrs={
            'long_name': 'detected fog times',
            'comment': 'detected/not-detected (1/0)'
        }
    )

    clear_air_flag = xr.DataArray(
        clear_air_flag,
        dims=('time',),
        coords={'time': ds['time']},
        attrs={
            'long_name': 'detected fog/clouds/precipitation',
            'comment': 'clear air/unclear air (0/1)'
        }
    )

    layers = xr.DataArray(
        layers,
        dims=('time', 'detected_layer'),
        coords={'time': ds['time'], 'detected_layer': np.arange(layers.shape[1])},
        attrs={
            'long_name': 'Detected Layers',
            'units': 'meters'
        }
    )


    ds['col_wt'] = col_wt
    ds['row_wt'] = row_wt
    ds['remnant'] = remnant
    ds['clouds'] = clouds
    ds['precip'] = precip
    ds['fog_flags'] = fog_flags
    ds['precip_flags'] = precip_flags
    ds['cloud_flags'] = cloud_flags
    ds['clear_air_flag'] = clear_air_flag
    ds['threshold_3hr'] = cleaned_thresh
    ds['layers'] = layers
    ds['overlap_function'] = overlap_function

    # get date
    date = ds['time'].idxmin(dim='time').dt.strftime("%Y%m%d").values

    ds.to_netcdf(f'/nfs/gce/globalscratch/dwefer/cl61_PBL/cl61_edges_{date}.nc')
    print(f'completed {date}')

In [6]:
def atwt2d(data2d, max_scale=-1):
    """
    Computes a trous wavelet transform (ATWT). Computes ATWT of the 2d array
    up to max_scale. If max_scale is outside the boundaries, number of scales
    will be reduced.

    Data is mirrored at the boundaries. 'Negative WT are removed. Not tested
    for non-square data.

    @authors: Bhupendra Raut and Dileep M. Puranik
    @references: Press et al. (1992) Numerical Recipes in C.

    Parameters:
    ===========
    data2d: ndarray
        2D image as array or matrix.
    max_scale:
        Computes wavelets up to max_scale. Leave blank for maximum possible
        scales.

    Returns:
    ========
    row_wt: ndarray (scales, ny, nx)
        Wavelet extracted by row-wise differencing (original – row-smoothed).
    col_wt: ndarray (scales, ny, nx)
        Wavelet extracted by column-wise differencing (row-smoothed – fully-smoothed).
    """
    if not isinstance(data2d, np.ndarray):
        sys.exit("the input is not a numpy array")

    # determine scales
    ny, nx = data2d.shape
    min_dim = min(ny, nx)
    max_possible = int(np.floor(np.log(min_dim) / np.log(2)))
    if max_scale < 0 or max_scale > max_possible:
        max_scale = max_possible

    # allocate output arrays
    total_wt = np.zeros((max_scale, ny, nx))
    row_wt   = np.zeros((max_scale, ny, nx))
    col_wt   = np.zeros((max_scale, ny, nx))

    # working buffers
    temp1 = np.zeros_like(data2d)
    temp2 = np.zeros_like(data2d)

    # scaling function coefficients
    sf = (0.0625, 0.25, 0.375)

    for scale in range(1, max_scale + 1):
        x1 = 2**(scale - 1)
        x2 = 2 * x1

        # --- row-wise smoothing into temp1 ---
        for i in range(nx):
            prev2 = abs(i - x2)
            prev1 = abs(i - x1)
            next1 = i + x1
            next2 = i + x2

            # mirror at edges
            if next1 > nx - 1:
                next1 = 2*(nx - 1) - next1
            if next2 > nx - 1:
                next2 = 2*(nx - 1) - next2
            if prev1 < 0 or prev2 < 0:
                prev1, prev2 = next1, next2

            for j in range(ny):
                left2  = data2d[j, prev2]
                left1  = data2d[j, prev1]
                right1 = data2d[j, next1]
                right2 = data2d[j, next2]
                temp1[j, i] = (
                    sf[0]*(left2 + right2) +
                    sf[1]*(left1 + right1) +
                    sf[2]* data2d[j, i]
                )

        # row-wise wavelet = original – row-smoothed
        row_wt[scale-1] = data2d - temp1

        # --- column-wise smoothing into temp2 ---
        for i in range(ny):
            prev2 = abs(i - x2)
            prev1 = abs(i - x1)
            next1 = i + x1
            next2 = i + x2

            if next1 > ny - 1:
                next1 = 2*(ny - 1) - next1
            if next2 > ny - 1:
                next2 = 2*(ny - 1) - next2
            if prev1 < 0 or prev2 < 0:
                prev1, prev2 = next1, next2

            for j in range(nx):
                top2    = temp1[prev2, j]
                top1    = temp1[prev1, j]
                bottom1 = temp1[next1, j]
                bottom2 = temp1[next2, j]
                temp2[i, j] = (
                    sf[0]*(top2 + bottom2) +
                    sf[1]*(top1 + bottom1) +
                    sf[2]* temp1[i, j]
                )

        # column-wise wavelet = row-smoothed – fully-smoothed
        col_wt[scale-1] = temp1 - temp2
        # total wavelet = original – fully-smoothed
        total_wt[scale-1] = data2d - temp2

        # prepare for next scale
        data2d[:] = temp2

    row_wt = np.where(row_wt > 0, row_wt, 0)
    col_wt = np.where(col_wt > 0, col_wt, 0)
    return row_wt, col_wt

def get_wavelets(ds_copy, ranges, time, varname='beta_att'):
    '''
    Computes a trous wavelet transform of the attenuated backscatter coefficient

    Parameters
    ===========
    ds_copy: xarray.Dataset
        Copy of the dataset containing the variable to be transformed
    ranges: ndarray
        1D array of range values corresponding to the vertical dimension
    time: ndarray
        1D array of time values corresponding to the time dimension
    varname: str
        Name of the variable in the dataset to be transformed (default is 'beta_att')
    
    Returns
    ===========
    row_wt: xarray.DataArray
        Row-wise a trous wavelet transform of the attenuated backscatter coefficient
    col_wt: xarray.DataArray
        Column-wise a trous wavelet transform of the attenuated backscatter coefficient
    remnant: xarray.DataArray
        Remnant of the 8-scale a trous wavelet transform of the attenuated backscatter
        coefficient
    '''
    remnant = ds_copy[varname].T.values.copy()
    row_wt, col_wt = atwt2d(remnant, 8)


    col_wt = xr.DataArray(
        col_wt,
        dims=('scale', 'range', 'time'),
        coords={'scale': range(1,9), 'range': ranges, 'time': time},
        attrs={
            'long_name': 'column-wise a trous wavelet transform of attenuated backscatter coefficient',
            'units': ds_copy[varname].units
        }
    )
    row_wt = xr.DataArray(
        row_wt,
        dims=('scale', 'range', 'time'),
        coords={'scale': range(1,9), 'range': ranges, 'time': time},
        attrs={
            'long_name': 'row-wise a trous wavelet transform of attenuated backscatter coefficient',
            'units': ds_copy[varname].units
        }
    )
    remnant = xr.DataArray(
        remnant,
        dims=('range', 'time'),
        coords={'range': ranges, 'time': time,},
        attrs={
            'long_name': 'remnant of 8-scale a trous wavelet transform of attenuated backscatter coefficient',
            'units': ds_copy[varname].units
        }
    )
    return row_wt, col_wt, remnant

def split_and_thresh(combined, resample_time="3h"):  
    """
    Splits the combined data into 3-hour blocks and applies automated OTSU thresholding on each block, returning the combined thresholded result.
    Parameters
    ----------
    combined : xarray.DataArray
        2D array with dimensions (range, time) containing the combined data to be thresholded.
    Returns
    -------
    result : np.ndarray
        2D array with dimensions (range, time) containing the thresholded data.
    """

    times = combined.time.values                      # shape (1440,)
    n_range, n_time = combined.shape
    result   = np.zeros((n_range, n_time), dtype=np.uint8)
    time_idx = pd.Index(times)
    for _, block in combined.resample(time=resample_time):
        arr = block.values.astype(np.float32)  
        block_normalized = cv2.normalize(arr, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        smooth_block = cv2.morphologyEx(block_normalized, cv2.MORPH_OPEN, kernel)
        _,th = cv2.threshold(smooth_block,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        locs = time_idx.get_indexer(block.time.values)
        result[:, locs] = th

    return result

def get_thresh(remnant, row_wt, col_wt, max_height=5000, resample_time="3h"):
    row_wt = row_wt.where(row_wt['range'] <= max_height, drop=True)
    col_wt = col_wt.where(col_wt['range'] <= max_height, drop=True)
    remnant = remnant.where(remnant['range'] <= max_height, drop=True)
    combined = (row_wt[2] + row_wt[3] + col_wt[4] + col_wt[5] + col_wt[6] + col_wt[7] + remnant).rename("combined")

    thresh = split_and_thresh(combined, resample_time)
    return thresh




def get_precip(backscatter, ranges, times, min_size=500, cloud_threshold=-5.5,
               surface_tol=10, min_extent=500, time_tol='10min'):
    """
    Identify precipitation regions in the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    min_size : int, optional
        Minimum size of a region to be considered, by default 500.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 10.
    min_extent : int, optional
        Minimum vertical extent of a region to be considered, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    Returns
    -------
    clean_labels : 2D ndarray, shape (n_range, n_time)
        Binary mask where 1 indicates the presence of a precipitation region and 0 indicates clear air.
    """

    water_mask   = backscatter > cloud_threshold
    water_mask   = remove_small_objects(water_mask, min_size=min_size)
    labels       = label(water_mask, connectivity=2)

    height_coords = ranges    # shape (n_range,)
    pd_times      = pd.to_datetime(times)
    tol_td        = pd.to_timedelta(time_tol)

    near_surface  = np.where(height_coords <= surface_tol + ranges.min())[0]

    clean_labels = labels.copy()

    # loop through regions
    for region in regionprops(labels):
        rid    = region.label
        coords = region.coords           # (N,2): (row, col)
        rows   = coords[:,0]
        cols   = coords[:,1]

        # find columns where the region has vertical_extent >= min_extent
        large_cols = []
        for c in np.unique(cols):
            rows_c = rows[cols == c]
            h_min  = height_coords[rows_c.min()]
            h_max  = height_coords[rows_c.max()]
            if (h_max - h_min) >= min_extent:
                large_cols.append(c)

        # remove a region if it doesn't extend far enough vertically
        if not large_cols:
            clean_labels[labels == rid] = 0
            continue

        # build a mask of times to keep
        large_times = pd_times[large_cols]
        keep_times  = np.array([any(abs(t - large_times) <= tol_td) for t in pd_times])  # nasty one liner, checks for all the times when the region was tall enough,
                                                                                        # and marks true if a time t is within tol_td of any of those times

        # remove any pixels outside the keep window
        bad = np.isin(cols, np.where(~keep_times)[0])
        bad_rows, bad_cols = rows[bad], cols[bad]
        clean_labels[bad_rows, bad_cols] = 0

        # check if it touches the near-surface region 
        touches_surface = np.any(np.isin(rows, near_surface))
        if not touches_surface:
            clean_labels[labels == rid] = 0

    return clean_labels


def get_precip_flags(backscatter, ranges, times, min_size=500, cloud_threshold=-5.5, surface_tol=10, min_extent=500, time_tol='10min'):
    """ 
    Get precipitation flags from the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    min_size : int, optional
        Minimum size of a region to be considered, by default 500.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 10.
    min_extent : int, optional
        Minimum vertical extent of a region to be considered, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    Returns
    -------
    flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of precipitation.
    """    
    precip     = get_precip(backscatter, ranges, times, min_size=min_size, cloud_threshold=cloud_threshold,surface_tol=surface_tol, min_extent=min_extent, time_tol=time_tol)
    valid_rows = np.where(ranges <= surface_tol)[0]
    flags      = (precip[valid_rows, :] > 0).any(axis=0).astype(np.uint8)
    return flags    
    

def get_clouds(backscatter, ranges, max_height=10000, cloud_threshold=-5.5, surface_tol=50):
    """
    Identify cloud regions in the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    max_height : int, optional
        The maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 50.
    Returns
    ------- 
    valid_labels : 2D ndarray, shape (n_range, n_time) 
        Binary mask where 1 indicates the presence of a cloud region and 0 indicates clear air.
    """
    near_surface = np.where(ranges <= surface_tol + ranges.min())[0]
    max_idx      = np.where(ranges <= max_height + ranges.min())[0].max()

    blurred_att  = cv2.GaussianBlur(backscatter.astype(np.float32), (5, 5), 0)

    water_mask   = (blurred_att > cloud_threshold)
    water_mask   = remove_small_objects(water_mask, min_size=500)

    labels       = label(water_mask, connectivity=2)
    valid_labels = labels.copy()

    for region in regionprops(labels):
        rows = region.coords[:, 0]

        # check if it touches the near-surface region (within 10m)
        touches_surface = np.any(np.isin(rows, near_surface))
        under_ceiling   = rows.min() <= max_idx

        if touches_surface or not under_ceiling:            
            valid_labels[labels == region.label] = 0
    
    return valid_labels


def get_cloud_base_height(backscatter, ranges, max_height=10000, cloud_threshold=-5.5, surface_tol=5):
    """
    For each time‐column, find the lowest (smallest) row‐index
    where a cloud region is present.
    
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)   
        The height (range) values corresponding to each gate index.
    max_height : int, optional 
        The maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 5.
    Returns
    -------
    first : 1D ndarray, shape (n_time,)
        1D array where each element is the index of the lowest cloud base height for that time.
        If no cloud is detected, the value is NaN.
    """
    labels  = get_clouds(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)

    present = labels > 0              # boolean mask
    # argmax gives first True index, but 0 if none are True
    first = np.argmax(present, axis=0).astype(float)
    # detect columns with no region
    no_region = ~present.any(axis=0)
    first[no_region] = np.nan
    return first


def get_cloud_flags(backscatter, ranges, max_height=10000, cloud_threshold=-5.5, surface_tol=5):
    """
    Get cloud flags from the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    max_height : int, optional
        The maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 5.
    Returns
    -------
    flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of clouds.
    """
    labels  = get_clouds(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    present = (labels > 0).any(axis=0)
    flags   = present.astype(np.uint8)
    return flags


def get_fog_flags(backscatter, ranges, times, surface_tol=5, cloud_threshold=-5.5, fog_threshold=-4.75, min_size=500, min_extent=500, time_tol='10min'):
    """
    Get fog flags from the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    surface_tol : int, optional
        Tolerance for surface height, by default 5.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    min_size : int, optional
        Minimum size of a region to be considered, by default 500.
    min_extent : int, optional
        Minimum vertical extent of a region to be considered by the precip flag, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    
    Returns
    -------
    flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of fog
    """

    blur_kernel = (5, 5)
    sigma_x = 0
    near_surface = np.where(ranges <= surface_tol + ranges.min())[0]
    blurred_att  = cv2.GaussianBlur(backscatter.astype(np.float32), blur_kernel, sigma_x)

    water_mask   = (blurred_att > fog_threshold)
    water_mask   = remove_small_objects(water_mask, min_size=500)
    precip_flags = get_precip_flags(backscatter, ranges, times, surface_tol=surface_tol, min_size=min_size, min_extent=min_extent, time_tol=time_tol, cloud_threshold=cloud_threshold)

    labels = label(water_mask, connectivity=2)
    
    n_time = labels.shape[1]
    flags  = np.zeros(n_time, dtype=np.uint8)

    # 6) any region that touches the near surface → fog
    for region in regionprops(labels):
        rows = region.coords[:,0]   # range‐gate indices
        cols = region.coords[:,1]   # time indices

        if np.any(np.isin(rows, near_surface)):
        
            flags[np.unique(cols)] = 1

    nan_times = np.isnan(backscatter[near_surface, :]).any(axis=0)
    flags[nan_times] = 1

    flags[precip_flags == 1] = 0 

    return flags


def remove_noisy_regions(backscatter, thresh, ranges, times, max_height=5000, cloud_threshold=-5.5, surface_tol = 5):
    """"
    Remove any regions that touch the top of the image and don't contain a cloud base .

    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    thresh : np.ndarray
        Binary mask of the thresholded image.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    max_height : int, optional
        The maximum height to consider for region removal, by default 5000.
    cloud_threshold : float, optional
        The threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : int, optional
        Tolerance for surface height, by default 5.
    Return
    -------
    cleaned : 2D uint8 array
        Copy of 'thresh' with the noisy regions removed.
    """

    max_idx        = np.where(ranges <= max_height)[0].max()     # maximum index of range that's below max height
    cleared_thr    = remove_small_objects(thresh > 0, min_size=900)      # cleans the threshold
    labels         = label(cleared_thr, connectivity=2)
    precip_flags   = get_precip_flags(backscatter, ranges, times, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    cloud_flags    = get_cloud_flags(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    combined_flags = precip_flags | cloud_flags     # I might experiment with using cloud base heights and precip flags separately, but this might work for now
    
    valid_labels = labels.copy()
    
    for region in regionprops(labels):
        rows = region.coords[:, 0]
        cols = region.coords[:, 1]

        touches_ceiling = (rows.max() >= max_idx)
        cloudy_or_rainy = np.any(combined_flags[cols])
        small_blob = region.area < 2000

        if (touches_ceiling or small_blob) and not cloudy_or_rainy:            
            valid_labels[labels == region.label] = 0
    

    valid_labels_da = xr.DataArray(
        valid_labels.astype(np.uint8),
        dims=('range', 'time'),
        coords={'time': times, 'range': ranges[ranges <= max_height]},
        attrs={
            'long_name': '3-hourly thresholds using OTSU method'
        }
    )

    return valid_labels_da




def get_clear_air_flag(backscatter, ranges, times, min_rain_size=500, min_rain_extent=500, time_tol='30min', max_height=10000, cloud_threshold=-5.5, fog_threshold = -4.75, surface_tol=5):
    '''
    1D ndarray where 0 is clear air and 1 is not clear air.

    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    min_rain_size : int, optional
        Minimum size of a rain region to be considered, by default 500.
    min_rain_extent : int, optional
        Minimum vertical extent of a rain region to be considered, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    max_height : int, optional
        Maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        Threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : float, optional
        Tolerance for surface height, by default 5.

    Returns
    -------
    combined_flag : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of fog
    '''
    fog    = get_fog_flags(backscatter, ranges, times, min_size=min_rain_size, min_extent=min_rain_extent, time_tol=time_tol, cloud_threshold=cloud_threshold, fog_threshold=fog_threshold, surface_tol=surface_tol)
    clouds = get_cloud_flags(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    precip = get_precip_flags(backscatter, ranges, times, min_size=min_rain_size, min_extent=min_rain_extent, cloud_threshold=cloud_threshold, surface_tol=surface_tol, time_tol=time_tol)


    combined_flag = (
        (fog == 1) | (clouds == 1) | (precip == 1)
    )

    return combined_flag.astype(np.uint8)



def get_layers(cleaned_regions, ranges, tolerance = 5, max_height = 5000):
    '''
    Parameters
    ----------
    mask : bool ndarray, shape (ntime, n_range, n_ray)
        True wherever you have a thresholded return.
    ranges : 1D float array, shape (n_range,)
        The height (range) values corresponding to each gate index.
    surface_tol : float
        Discard any edge at or below this height (e.g. ground clutter).

    Returns
    -------
    layers : float64 ndarray, shape (ntime, max_edges)
        Each row t contains, in blob order:
          [ bottom₁, top₁, bottom₂, top₂, … ]
        for all regions at time t.  Rows are padded with NaN out to
        the maximum number of edges seen in any time slice.
    '''
    max_regions = int(cleaned_regions.max())  # maximum number of regions in any time slice
    max_layers  = max_regions * 2  # each region has a bottom and top edge, so we double the number of regions
    n_time      = cleaned_regions.shape[1]
    layers      = np.full((n_time, max_layers), np.nan, dtype=np.float64) 
    foorprint   = np.ones(50, dtype=bool)
    
    for t in range(n_time):
        # label connected blobs in this time slice
        profile_1d = cleaned_regions[:, t]
        profile_1d = binary_closing(profile_1d, footprint=foorprint)  # close small gaps
        profile_2d = profile_1d[:, np.newaxis]
        lbl = label(profile_2d, connectivity=1)
        props = regionprops(lbl)
        idx = 0

        for region in props[:max_layers]:
            rows     = region.coords[:, 0]
            bottom_h = ranges[rows.min()]
            top_h    = ranges[rows.max()]

            if bottom_h <= tolerance + ranges.min():
                if top_h > tolerance and idx < max_layers:  # ignore edges below the surface tolerance
                    layers[t, idx] = top_h
                    idx += 1
                    
            elif top_h >= max_height - tolerance:
                if bottom_h > tolerance and idx < max_layers:
                    layers[t, idx] = bottom_h
                    idx += 1

            else:
                if bottom_h > tolerance and idx < max_layers:
                    layers[t, idx] = bottom_h
                    idx += 1

                if top_h > tolerance and idx < max_layers:
                    layers[t, idx] = top_h
                    idx += 1
            
            if idx >= max_layers:
                break
            
    return layers


def get_clouds_and_precip(backscatter, ranges, times,  min_rain_size=500, min_rain_extent=2000, time_tol='10min', max_height=10000, cloud_threshold=-5.5, surface_tol=5):
    """
    Get regions of interest from the attenuated backscatter data.
    
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    min_rain_size : int, optional
        Minimum size of a rain region to be considered, by default 500.
    min_rain_extent : int, optional
        Minimum vertical extent of a rain region to be considered, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    max_height : int, optional
        Maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        Threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : float, optional
        Tolerance for surface height, by default 5.

    Returns
    -------
        precip_regions : 2D ndarray, shape (n_range, n_time)
            Binary mask where 1 indicates the presence of a precipitation region and 0 indicates clear air
        cloud_regions : 2D ndarray, shape (n_range, n_time)
            Binary mask where 1 indicates the presence of a cloud region and 0 indicates clear air
    """
    precip_regions = get_precip(backscatter, ranges, times, min_size=min_rain_size, min_extent=min_rain_extent, time_tol=time_tol, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    cloud_regions = get_clouds(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    
    return cloud_regions, precip_regions


def get_flags(backscatter, ranges, times, min_rain_size=500, min_rain_extent=2000, time_tol='10min', max_height=10000, cloud_threshold=-5.5, fog_threshold = -4.75, surface_tol=5):
    """
    Get flags for clouds, precipitation, and fog from the attenuated backscatter data.
    Parameters
    ----------
    backscatter : 2D ndarray, shape (n_range, n_time)
        The attenuated backscatter coefficient data.
    ranges : 1D ndarray, shape (n_range,)
        The height (range) values corresponding to each gate index.
    times : 1D ndarray, shape (n_time,)
        The time values corresponding to each column in the backscatter data.
    min_rain_size : int, optional
        Minimum size of a rain region to be considered, by default 500.
    min_rain_extent : int, optional
        Minimum vertical extent of a rain region to be considered, by default 500.
    time_tol : str, optional
        Time tolerance for considering a region as rain, by default '10min'.
    max_height : int, optional
        Maximum height to consider for cloud detection, by default 10000.
    cloud_threshold : float, optional
        Threshold for cloud/fog/rain detection, by default -5.5.
    surface_tol : float, optional
        Tolerance for surface height, by default 5.
    Returns
    -------
    fog_flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of fog
    cloud_flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of clouds
    precip_flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of precipitation
    clear_air_flags : 1D ndarray, shape (n_time,)
        1D array where 0 indicates clear air and 1 indicates the presence of any
        cloud, fog, or precipitation
    """
    fog_flags = get_fog_flags(backscatter, ranges, times, min_size=min_rain_size, min_extent=min_rain_extent, time_tol=time_tol, cloud_threshold=cloud_threshold, fog_threshold=fog_threshold, surface_tol=surface_tol)
    cloud_flags = get_cloud_flags(backscatter, ranges, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)
    precip_flags = get_precip_flags(backscatter, ranges, times, min_size=min_rain_size, min_extent=min_rain_extent, cloud_threshold=cloud_threshold, surface_tol=surface_tol, time_tol=time_tol)
    clear_air_flags = get_clear_air_flag(backscatter, ranges, times, min_rain_size=min_rain_size, min_rain_extent=min_rain_extent, time_tol=time_tol, max_height=max_height, cloud_threshold=cloud_threshold, surface_tol=surface_tol)

    return fog_flags.astype(np.uint8), cloud_flags.astype(np.uint8), precip_flags.astype(np.uint8), clear_air_flags.astype(np.uint8)

In [5]:
file_list = glob.glob("")

print(f'file list: {file_list}')

for i, fname in enumerate(file_list):
    ds = xr.open_dataset(fname)
    print(f'opened {fname}')
    ds.sortby('time')
    # Variables to correct
    variables = ['beta_att', 'p_pol', 'x_pol', 'linear_depol_ratio']

    ds['linear_depol_ratio_uncorrected'] = ds['x_pol'] / (ds['x_pol'] + ds['p_pol'])
    
    for var in variables:
        if var != 'linear_depol_ratio':
                ds = act.corrections.correct_ceil(ds, var_name=var)
    
    # Compute linear depolarization ratio
    ds["linear_depol_ratio_manual"] = ds["x_pol"] / (ds["x_pol"] + ds["p_pol"])

    overlap_function = ds['overlap_function']
    ds = ds.drop_vars(['overlap_function'])  # This gets added back in create_file, it is temporarily removed to allow me to drop nans prior to running algorithm
    
    
    ds = ds.dropna(dim='range', how='all') \
       .dropna(dim='time',  how='all')
    
    max_height=5000
    ds_copy = ds.copy(deep=True)
    row_wt, col_wt, remnant = get_wavelets(ds_copy, ds['range'].values, ds['time'].values)
    ds_copy.close()
    print(f'got wavelets from {fname}')
    clouds, precip = get_clouds_and_precip(ds['beta_att'].T.values, ds['range'].values, ds['time'].values)
    fog_flags, precip_flags, cloud_flags, clear_air_flag = get_flags(ds['beta_att'].T.values, ds['range'].values, ds['time'].values, max_height=max_height)
    thresh = get_thresh(remnant, row_wt, col_wt, max_height=max_height)
    cleaned_thresh = remove_noisy_regions( ds['beta_att'].T.values, thresh, ds['range'].values, ds['time'].values)
    cleaned_thresh = cleaned_thresh.where(cleaned_thresh.range <= max_height, drop=True)
    layers = get_layers(cleaned_thresh.values, cleaned_thresh['range'].values)
    create_file(ds, row_wt, col_wt, remnant, clouds, precip, fog_flags, precip_flags, cloud_flags, clear_air_flag, cleaned_thresh, layers, overlap_function)
    print(f'created file for {fname}, file number {i+1}/{len(file_list)}')

file list: []
